In [412]:
import numpy as np
import tensorflow as tf

def positional_encoding(dim, sentence_length):
    encoded_vec = np.array([pos/ np.power(10000, 2*i/ dim) for pos in range(sentence_length) for i in range(dim)])
    encoded_vec[::2] = np.sin(encoded_vec[::2])
    encoded_vec[1::2] = np.cos(encoded_vec[1::2])
    return tf.constant(encoded_vec.reshape([sentence_length, dim]), dtype=tf.float32)

In [414]:
def layer_norm(input, eps=1e-6):
    feature_shape = input.get_shape()[-1:]
    mean = tf.keras.backend.mean(input, [-1], keepdims=True)
    std = tf.keras.backend.std(input, [-1], keepdims=True)
    beta = tf.Variable(tf.zeros(feature_shape), trainable=False)
    gamma = tf.Variable(tf.ones(feature_shape), trainable=False)
    return gamma * (input - mean) /  (std+eps) + beta

In [415]:
def sublayer_connection(input, sublayer, dropout=0.2):
    outputs = layer_norm(input + tf.keras.layers.Dropout(dropout)(sublayer))
    return outputs

In [416]:
def scaled_dot_product_attention(query, key, value, masked=False):
    key_dim_size = float(key.get_shape().as_list()[-1])
    key = tf.transpose(key, perm=[0,2,1])
    
    outputs = tf.matmul(query, key) / tf.sqrt(key_dim_size)
    
    if masked : 
        diag_vals = tf.ones_like(outputs[0,:,:])
        tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()
        masks = tf.tile(tf.expand_dims(tril,0), [tf.shape(outputs)[0],1,1])
        paddings = tf.ones_like(masks)*(-2**30)
        outputs = tf.where(tf.equal(masks,0), paddings, outputs)
        
    attention_map = tf.nn.softmax(outputs)
    return tf.matmul(attention_map, value)

In [417]:
def multi_head_attention(query, key, value, num_units, heads, masked=False):
    query = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(query)
    key = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(key)
    value = tf.keras.layers.Dense(num_units, activation= tf.nn.relu)(value)
    
    query = tf.concat(tf.split(query, heads, axis=-1), axis=0)
    key = tf.concat(tf.split(key, heads, axis=-1), axis=0)
    value = tf.concat(tf.split(value, heads, axis=-1), axis=0)
    
    attention_map = scaled_dot_product_attention(query, key, value, masked)
    attn_outputs = tf.concat(tf.split(attention_map, heads, axis=0), axis=-1)
    attn_outputs = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(attn_outputs)
    
    return attn_outputs

In [418]:
def feed_forward(input, num_units):
    feature_shape = input.get_shape()[-1]
    inner_layer = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(input)
    outputs = tf.keras.layers.Dense(feature_shape)(inner_layer)
    return outputs

In [419]:
def encoder_module(input, model_dim, ffn_dim, heads):
    self_attn = sublayer_connection(input, multi_head_attention(input, input, input, model_dim,heads))
    outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
    return outputs

def encoder(input, model_dim, ffn_dim, heads, num_layers):
    outputs = input
    for i in range(num_layers):
        outputs = encoder_module(outputs, model_dim, ffn_dim, heads)
        
    return outputs

In [420]:
def decoder_module(input, encoder_outputs, model_dim, ffn_dim, heads):
    masked_self_attn = sublayer_connection(input, multi_head_attention(input, input, input, model_dim, heads, masked=True))
    
    self_attn = sublayer_connection(masked_self_attn, multi_head_attention(masked_self_attn, encoder_outputs, encoder_outputs, model_dim, heads))
    
    outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
    return outputs

def decoder(input, encoder_outputs, model_dim, ffn_dim, heads, num_layers):
    outputs= input
    for i in range(num_layers):
        outputs = decoder_module(outputs, encoder_outputs, model_dim, ffn_dim, heads)
        
    return outputs

In [421]:
!pip install konlpy

You should consider upgrading via the 'C:\ProgramData\Anaconda3\envs\py37\python.exe -m pip install --upgrade pip' command.


In [423]:
import re
import tensorflow as tf

filters = "([~.,!?\"':;)(])"
PAD = '<PADDING>'
STD = '<START>'
END = '<END>'
UNK = '<UNKNOWN>'

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER= [PAD,STD,END, UNK]
CHANGE_FILTER = re.compile(filters)

In [424]:
from sklearn.model_selection import train_test_split
import pandas as pd

def load_data(data_path):
    data_df = pd.read_csv(data_path, header=0)
    quetion, answer = list(data_df['Q']), list(data_df['A'])
    train_input, eval_input, train_label, eval_label = train_test_split(quetion, answer, test_size=0.33, random_state=111)
    
    return train_input, train_label, eval_input, eval_label

In [426]:
def load_vocabulary(data_path):
    data_df = pd.read_csv(data_path, encoding='utf-8')
    quetion, answer = list(data_df['Q']), list(data_df['A'])
    if tokenize_as_morph:
        question = prepro_like_morphlized(quetion)
        answer = prepro_like_morphlized(answer)
        
    data = []
    data.extend(question)
    data.extend(answer)
    words = data_tokenizer(data)
    words = list(set(words))
    words[:0] = MARKER
    
    char2idx = {char:idx for idx, char in enumerate(words)}
    idx2char = {idx:char for idx, char in enumerate(words)}
    return char2idx, idx2char, len(char2idx)


In [430]:
from konlpy.tag import Okt

def prepro_like_morphlized(data):
    morph_analyzer = Okt()
    result_data = list()
    for seq in data:
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphlized_seq)
        
    return result_data

In [431]:
def data_tokenizer(data):
    words = []
    for sentence in data:
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    return [word for word in words if word]

In [432]:
import numpy as np

def enc_processing(value, dictionary):
    sequences_input_index =[]
    sequences_length = []
    
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
        
    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = []
        for word in sequence.split():
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            else:
                sequence_index.extend([dictionary[UNK]])
        if len(sequence_index) > max_len:
            sequence_index = sequence_index[:max_len]
        sequences_length.append(len(sequence_index))
        sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
        sequences_input_index.append(sequence_index)
    return np.asarray(sequences_input_index), sequences_length

In [433]:
def dec_output_processing(value, dictionary):
    sequences_output_index =[]
    sequences_length = []
    
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
        
    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = []
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]
        if len(sequence_index) > max_len:
            sequence_index = sequence_index[:max_len]
        sequences_length.append(len(sequence_index))
        sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
        sequences_output_index.append(sequence_index)
    return np.asarray(sequences_output_index), sequences_length

In [434]:
def dec_target_processing(value, dictionary):
    sequences_target_index = []

    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
        
    for sequence in value:
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        sequence_index = [dictionary[word] for word in sequence.split()]
        if len(sequence_index) >= max_len:
            
            sequence_index = sequence_index[:max_len - 1] + [dictionary[END]]
        else:
            sequence_index +=[dictionary[END]]
        sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
        sequences_target_index.append(sequence_index)
    return np.asarray(sequences_target_index)

In [435]:
def train_input_fn(train_input_enc, train_output_enc, train_target_dec, batch_size):
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices((train_input_enc, train_output_enc, train_target_dec))
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(rearrange)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn(eval_input_enc, eval_output_enc, eval_target_dec, batch_size):
    dataset = tf.compat.v1.data.Dataset.from_tensor_slices((eval_input_enc, eval_output_enc, eval_target_dec))
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(rearrange)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def rearrange(input, output, target):
    features = {'input':input, 'output':output}
    return features, target

In [436]:
def pred2string(value, dictionary):
    sentence_string = []
    is_finished = False
    
    for v in value:
        sentence_string = [dictionary[index] for index in v['indexs']]
        
    answer = ""
    for word in sentence_string:
        if word == END:
            if_finished = True
            break
        if word != PAD and word != END:
            answer += word
            answer += " "
            
    return answer, is_finished

In [437]:
import pandas as pd 

tokenize_as_morph = True

data_path = 'https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv'

char2idx, idx2char, len_vocab = load_vocabulary(data_path)
train_input, train_label, eval_input, eval_label = load_data(data_path)

In [438]:
def model(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    position_encode = positional_encoding(params['embedding_size'], params['max_len'])
    if params['xavier_initializer']:
        embedding_initializer = 'glorot_normal'
    else: 
        embedding_initializer = 'uniform'
        
    embedding = tf.keras.layers.Embedding(params['len_vocab'], params['embedding_size'], embeddings_initializer=embedding_initializer)
    
    x_embedded_matrix = embedding(features['input']) + position_encode
    y_embedded_matrix = embedding(features['output']) + position_encode
    
    encoder_outputs = encoder(x_embedded_matrix, params['model_hidden_size'], params['ffn_hidden_size'], params['attention_head_size'], params['layer_size'])
    decoder_outputs = decoder(y_embedded_matrix, encoder_outputs, params['model_hidden_size'], params['ffn_hidden_size'], params['attention_head_size'], params['layer_size'])
    
    logits = tf.keras.layers.Dense(params['len_vocab'])(decoder_outputs)
    predict = tf.argmax(logits, 2)
    
    if PREDICT:
        predictions = {'indexs':predict,'logits':logits} 
        return tf.estimator.EstimatorSpec(mode, predictions= predictions)
    
    labels_= tf.one_hot(labels, params['len_vocab'])
    loss =tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))
    accuracy = tf.compat.v1.metrics.accuracy(labels=labels, predictions=predict)   
    
    metrics = {'accuracy' : accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    
    if EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    assert TRAIN
    
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(loss, global_step = tf.compat.v1.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op = train_op)

In [439]:
max_len = 25
epoch = 5000
batch_size =256
embedding_size = 100
model_hidden_size = 100
ffn_hidden_size = 100
attention_head_size = 100
lr= 0.001
layer_size =3
xavier_initializer = True

In [440]:
train_input_enc, train_input_enc_length = enc_processing(train_input, char2idx)
train_output_dec, train_output_dec_length = dec_output_processing(train_label, char2idx)
train_target_dec = dec_target_processing(train_label, char2idx)

eval_input_enc, eval_input_enc_length = enc_processing(eval_input, char2idx)
eval_output_dec, eval_output_dec_length = dec_output_processing(eval_label, char2idx)
eval_target_dec = dec_target_processing(eval_label, char2idx)

In [441]:
transformer = tf.estimator.Estimator(
    model_fn = model,
    params = {'embedding_size': embedding_size,
              'model_hidden_size': model_hidden_size,
              'ffn_hidden_size': ffn_hidden_size,
              'attention_head_size': attention_head_size,
              'learning_rate': lr,
              'len_vocab': len_vocab,
              'layer_size': layer_size,
              'max_len': max_len,
              'xavier_initializer': xavier_initializer}
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\bitcamp\\AppData\\Local\\Temp\\tmpf0jubh5d', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [442]:
transformer.train(input_fn=lambda: train_input_fn(train_input_enc, train_output_dec, train_target_dec, batch_size), steps=epoch)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 9.667584, step = 0
INFO:tensorflow:global_step/sec: 5.76189
INFO:tensorflow:loss = 1.6265355, step = 100 (17.355 sec)
INFO:tensorflow:global_step/sec: 7.02258
INFO:tensorflow:loss = 1.3423468, step = 200 (14.241 sec)
INFO:tensorflow:global_step/sec: 6.99627
INFO:tensorflow:loss = 1.1492085, step = 300 (14.292 sec)
INFO:tensorflow:global_step/sec: 6.98917
INFO:tensorflow:loss = 0.95866185, step = 400 (14.308 sec)
INFO:tensorflow:global_step/sec: 6.97828
INFO:tensorflow:loss = 0.828

In [443]:
eval_result = transformer.evaluate(input_fn=lambda: eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec, batch_size))
print("accuracy: {accuracy: 0.3f}".format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-04-03T19:44:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.86965s
INFO:tensorflow:Finished evaluation at 2022-04-03-19:44:33
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8650436, global_step = 5000, loss = 1.5579826
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
accuracy:  0.865


In [447]:
def chatbot(sentence):
    pred_input_enc, pred_input_enc_length = enc_processing([sentence], char2idx)
    pred_output_dec, pred_output_length = dec_output_processing([""], char2idx)
    pred_target_dec = dec_target_processing([""], char2idx)
    
    for i in range(max_len):
        if i > 0:
            pred_output_dec, pred_output_dec_length = dec_output_processing([answer], char2idx)
            pred_target_dec = dec_target_processing([answer], char2idx)
            
        predictions = transformer.predict(input_fn=lambda: eval_input_fn(pred_input_enc, pred_output_dec, pred_target_dec, 1)) 
        
        answer, finished = pred2string(predictions, idx2char)
        
        if finished:
           break
    
    return answer
            

In [448]:
chatbot("안녕?")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was fina

'안녕하세요 '

In [449]:
chatbot("너 누구냐?")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was fina

'저 랑 한 잔해 요 '

In [450]:
chatbot("뭐 먹었어?")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was fina

'저 랑 놀아요 '

In [454]:
chatbot("내일 날씨 어때?")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was fina

'날씨 어플 에 물어보세요 '

In [453]:
chatbot("이제 그만 잘래")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bitcamp\AppData\Local\Temp\tmpf0jubh5d\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was fina

'힘 은 버리세요 '